<a href="https://colab.research.google.com/github/diegovanelli/dataScience_estudos/blob/main/Notebooks/FormulaMagicaV2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Busca os dados no site fundamentus e coloca em um dataset

In [ ]:
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from unicodedata import normalize
import pandas as pd


url = 'https://fundamentus.com.br/resultado.php'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

try:
    req = Request(url, headers = headers)
    response = urlopen(req)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    

except HTTPError as e:
    print(e.status, e.reason)

except URLError as e:
    print(e.reason)

campos = soup.table.tr.findAll('a')
camposArr= []
for campo in campos:
  normalizar = normalize("NFD", campo.get_text()).encode("ascii", "ignore").decode("utf-8")
  camposArr.append(normalizar.lower().replace('/', '_').replace(' ', '_').replace('.', ''))

acoes = []
acao = {}
valores = soup.table.tbody.findAll('tr')
for valor in valores:
  valorLinhas = valor.findAll('td')
  for linha in range(len(camposArr)):
    if (linha == 0):
      campo = valorLinhas[linha].get_text()
    else:
      campo = float(valorLinhas[linha].get_text().replace('.', '').replace(',','.').replace('%', ''))
    acao[camposArr[linha]] = campo
  acoes.append(acao)
  acao = {}
  
dataset = pd.DataFrame(acoes)

!pip install sqlalchemy

from sqlalchemy import create_engine, MetaData, Table, inspect # adicionando o método inspect

engine = create_engine('sqlite:///:memory:')
engine
type(engine)

dataset.to_sql('acoes', engine)

inspector = inspect(engine) # criando um Inspector object
print(inspector.get_table_names()) # Exibindo as tabelas com o inspecto

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
['acoes']


In [ ]:
pasta = '/drive/MyDrive/BASE_DADOS_FORMULA_MAGICA/'

from datetime import date
from google.colab import drive
drive.mount ('/drive')
data_atual = date.today()
data_em_texto = data_atual.strftime('%d_%m_%Y')
dataset.to_excel(pasta + 'dataset_' + data_em_texto + '.xlsx')

Mounted at /drive


# Faz o rank do P/L

In [ ]:
# query_pl = 'select * from acoes where ev_ebit > 0 patrim_liq > 0 and liq2meses > 0 order by ev_ebit'
query_pl = 'select * from acoes order by ev_ebit'
filtro_pl = pd.read_sql(query_pl, engine)
filtro_pl['rank_pl'] = filtro_pl.index + 1
filtro_pl.to_sql('acoes_pl', engine)

In [ ]:
filtro_pl.head(10)

,index,papel,cotacao,p_l,p_vp,psr,divyield,p_ativo,p_capgiro,p_ebit,...,mrg_ebit,mrg_liq,liq_corr,roic,roe,liq2meses,patrim_liq,divbrut__patrim,cresc_rec5a,rank_pl
0,927,PTQS4,189.46,43.43,2.19,4.717,0.0,1.577,9.89,-7214.41,...,-0.07,11.37,3.32,-0.02,5.04,0.0,1.714950e+09,0.24,56.13,1
1,808,BMTO3,7.82,17.84,2.14,0.000,0.0,2.097,0.00,-2711.85,...,0.00,0.00,0.00,0.00,12.00,0.0,1.046320e+09,0.00,0.00,2
2,23,PITI4,1.28,-204.74,0.00,0.000,0.0,0.000,0.00,-2637.65,...,0.00,0.00,0.00,0.00,0.00,0.0,0.000000e+00,0.00,-29.63,3
3,799,BMTO4,7.57,17.27,2.07,0.000,0.0,2.030,0.00,-2625.16,...,0.00,0.00,0.00,0.00,12.00,0.0,1.046320e+09,0.00,0.00,4
4,630,RDTR3,6.30,8.11,1.37,0.000,0.0,1.320,0.00,-1919.75,...,0.00,0.00,0.00,0.00,16.95,0.0,4.970920e+08,0.00,0.00,5
5,271,TROR3,2.63,-0.03,-0.01,0.598,0.0,0.532,-0.01,-6.57,...,-9.11,-1745.52,0.01,-9.77,21.14,0.0,-1.959650e+09,-0.80,6.34,6
6,277,TROR4,1.56,-0.02,0.00,0.355,0.0,0.316,0.00,-3.90,...,-9.11,-1745.52,0.01,-9.77,21.14,0.0,-1.959650e+09,-0.80,6.34,7
7,156,CGOS3,26.20,-4.85,1.89,0.509,0.0,0.167,-0.83,-256.60,...,-0.20,-10.48,0.54,-0.08,-39.02,0.0,4.533870e+08,2.48,8.97,8
8,401,OSXB3,6.07,0.62,0.00,0.574,0.0,0.009,-0.03,-2.10,...,-27.31,-40.72,0.03,-0.46,-0.55,175245.0,-5.669510e+09,-0.88,30.85,9
9,20,RJCP3,0.01,-224.70,0.65,0.000,0.0,0.619,0.00,-522.94,...,0.00,0.00,0.00,0.00,-0.29,0.0,1.728720e+07,0.00,0.00,10


# Fazer rank do ROE

In [ ]:
query_roe = 'select * from acoes_pl order by roic desc'
filtro_roe = pd.read_sql(query_roe, engine)
filtro_roe['rank_roe'] = filtro_roe.index + 1

In [ ]:
filtro_roe.head(10)

,level_0,index,papel,cotacao,p_l,p_vp,psr,divyield,p_ativo,p_capgiro,...,mrg_liq,liq_corr,roic,roe,liq2meses,patrim_liq,divbrut__patrim,cresc_rec5a,rank_pl,rank_roe
0,186,397,TRPN3,0.71,0.44,0.54,0.263,104.74,0.399,0.59,...,59.29,3.61,523.01,122.35,0.0,5.935800e+07,0.00,5.77,187,1
1,136,160,ADHM3,1.56,-4.09,-1.29,770.924,0.00,105.126,-1.43,...,-18860.60,0.00,521.70,31.47,2064.0,-1.977600e+07,0.00,43.48,137,2
2,155,192,VTLM3,0.79,-2.07,-0.65,390.404,0.00,53.237,-0.72,...,-18860.60,0.00,521.70,31.47,0.0,-1.977600e+07,0.00,43.48,156,3
3,363,372,BPHA3,0.62,0.06,-0.24,-20.054,0.00,0.995,-0.38,...,-33652.50,0.06,401.01,-402.50,0.0,-2.922940e+08,-0.26,-53.39,364,4
4,411,683,BRBI11,14.91,10.24,2.07,1.096,5.42,0.289,2.10,...,10.70,1.16,243.06,20.18,1319420.0,7.575230e+08,5.91,0.00,412,5
5,606,732,MPLU3,26.80,12.20,17.37,6.877,0.00,2.676,50.65,...,56.37,1.06,129.57,142.40,0.0,2.502790e+08,0.00,-18.88,607,6
6,333,384,INEP4,1.21,0.19,-0.16,8.926,0.00,0.161,-0.25,...,4767.84,0.13,112.71,-83.71,3536990.0,-1.381600e+09,-0.57,-14.07,334,7
7,334,385,INEP3,1.34,0.21,-0.18,9.885,0.00,0.178,-0.27,...,4767.84,0.13,112.71,-83.71,10748200.0,-1.381600e+09,-0.57,-14.07,335,8
8,434,395,AGEN33,0.16,0.39,-0.03,0.000,0.00,0.066,-0.17,...,0.00,0.09,87.67,-8.85,0.0,-1.112630e+09,-1.11,-87.03,435,9
9,506,629,CMET4,3.91,8.10,5.38,3.068,0.00,3.136,9.47,...,43.03,2.64,81.40,66.40,0.0,2.847970e+09,0.18,30.18,507,10


# Fazer a soma do rank P/L com o ROE

In [ ]:
filtro_roe['total_rank'] = filtro_roe['rank_pl'] + filtro_roe['rank_roe']
filtro_roe.drop('level_0', axis='columns', inplace=True)
filtro_roe.drop('index', axis='columns', inplace=True)

# Resultado final

In [ ]:
filtro_roe.to_sql('acoes_final', engine)
# query_final = 'select * from acoes_final where ev_ebit > 0 and patrim_liq > 0 and roic > 0 and liq2meses > 0 order by total_rank'
query_final = 'select * from acoes_final where patrim_liq > 0 and roic > 0 and liq2meses > 0 order by total_rank'
filtro_final = pd.read_sql(query_final, engine)
filtro_final.drop('index', axis='columns', inplace=True)
#filtro_final.set_index(['papel'], inplace=True)

In [ ]:
filtro_final.to_excel(pasta + 'filtro_final_' + data_em_texto + '.xlsx')

In [ ]:
filtro_final.head(30)

,papel,cotacao,p_l,p_vp,psr,divyield,p_ativo,p_capgiro,p_ebit,p_ativ_circliq,...,liq_corr,roic,roe,liq2meses,patrim_liq,divbrut__patrim,cresc_rec5a,rank_pl,rank_roe,total_rank
0,PSSA3,24.09,17.32,1.66,0.647,4.45,0.339,5.72,0.79,-2.18,...,1.10,54.25,9.56,4.720400e+07,9.409950e+09,0.00,7.69,326,17,343
1,SYNE3,5.12,0.63,0.51,0.336,170.18,0.176,1.06,0.51,-0.97,...,4.43,40.55,79.85,2.219590e+06,1.543340e+09,0.81,36.81,341,29,370
2,GOAU3,10.83,2.14,0.72,0.137,6.93,0.150,0.56,0.57,1.62,...,2.26,33.86,33.85,9.131020e+05,1.625920e+10,0.77,19.54,339,45,384
3,GOAU4,11.53,2.28,0.77,0.145,6.50,0.160,0.60,0.61,1.73,...,2.26,33.86,33.85,1.047390e+08,1.625920e+10,0.77,19.54,340,46,386
4,TASA4,14.65,2.66,1.92,0.626,10.86,0.823,3.70,1.93,8.24,...,1.50,53.73,72.20,1.519300e+07,9.268330e+08,0.64,40.47,387,18,405
5,TASA3,14.80,2.68,1.94,0.632,10.75,0.832,3.74,1.95,8.32,...,1.50,53.73,72.20,4.781700e+05,9.268330e+08,0.64,40.47,389,19,408
6,BRBI11,14.91,10.24,2.07,1.096,5.42,0.289,2.10,7.17,2.28,...,1.16,243.06,20.18,1.319420e+06,7.575230e+08,5.91,0.00,412,5,417
7,UNIP3,96.50,4.10,3.11,1.301,15.24,1.458,4.81,3.45,-156.02,...,2.42,65.04,75.83,2.060740e+06,3.220550e+09,0.44,23.19,411,12,423
8,GGBR3,22.20,2.34,0.81,0.443,6.67,0.492,1.89,1.86,5.79,...,2.22,33.91,34.51,8.550110e+05,4.733320e+10,0.26,19.54,381,43,424
9,PETR4,27.15,2.01,0.95,0.574,61.29,0.374,15.06,1.24,-0.86,...,1.17,32.30,47.38,3.223350e+09,3.724980e+11,0.79,15.36,383,49,432


In [ ]:
df = filtro_final.filter(items=[1,2,3,6,7], axis=0)
df.set_index(['papel'], inplace=True)
df

,cotacao,p_l,p_vp,psr,divyield,p_ativo,p_capgiro,p_ebit,p_ativ_circliq,ev_ebit,...,liq_corr,roic,roe,liq2meses,patrim_liq,divbrut__patrim,cresc_rec5a,rank_pl,rank_roe,total_rank
papel,,,,,,,,,,,,,,,,,,,,,
SYNE3,5.45,0.68,0.54,0.357,159.87,0.187,1.13,0.54,-1.03,0.93,...,4.43,40.55,79.85,2120670.0,1.543340e+09,0.81,36.81,340,28,368
GOAU3,10.17,2.01,0.68,0.128,22.62,0.141,0.53,0.54,1.52,0.74,...,2.26,33.86,33.85,1138860.0,1.625920e+10,0.77,19.54,336,47,383
GOAU4,11.03,2.18,0.74,0.139,20.85,0.153,0.58,0.58,1.65,0.78,...,2.26,33.86,33.85,85723300.0,1.625920e+10,0.77,19.54,338,48,386
UNIP3,88.49,3.76,2.85,1.193,16.62,1.337,4.41,3.16,-143.07,2.93,...,2.42,65.04,75.83,2337290.0,3.220550e+09,0.44,23.19,404,12,416
BRBI11,14.74,10.12,2.04,1.084,5.48,0.286,2.08,7.09,2.25,3.19,...,1.16,243.06,20.18,1682740.0,7.575230e+08,5.91,0.00,414,5,419


In [ ]:
valor_investir = 1000
quantidade_de_acoes = 5

percentual = (100 / quantidade_de_acoes)/100
print(percentual * 100)


compra = df.head(quantidade_de_acoes).cotacao

valor_final = 0
quantidade_acoes = []
valor_acao_total = []
for i in range(len(compra)):
  quantidade_acoes.append(int((valor_investir * percentual) / compra[i]))
  valor_acao_total.append( compra[i] * int((valor_investir * percentual) / compra[i]))
  valor_final += compra[i] * int((valor_investir * percentual) / compra[i])

print(valor_final)
acoes_compra_final = pd.DataFrame({
    'acao': compra.index.values,
    'valor_papel': compra.values,
    'quantidade_acao': quantidade_acoes,
    'valor_total_acao' : valor_acao_total
})
acoes_compra_final.set_index('acao')

In [ ]:
acoes_compra_final.to_excel(pasta + 'compras_' + data_em_texto + '.xlsx')

In [ ]:
filtro_final = filtro_final.set_index("papel")

# 1. Importando as bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web

!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
yf.pdr_override()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
teste = web.get_data_yahoo("ABEV3.sa")

[*********************100%***********************]  1 of 1 completed


In [ ]:
teste


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-05,0.520882,0.520882,0.520882,0.520882,0.321626,985
2000-01-06,0.494478,0.494478,0.494478,0.494478,0.305322,227
2000-01-07,0.494478,0.494478,0.494478,0.494478,0.305322,151
2000-01-10,0.494478,0.494478,0.494478,0.494478,0.305322,1516
2000-01-11,0.494478,0.494478,0.494478,0.494478,0.305322,3791
...,...,...,...,...,...,...
2022-08-01,14.860000,14.890000,14.430000,14.630000,14.630000,34565900
2022-08-02,14.560000,14.790000,14.510000,14.710000,14.710000,12515800
2022-08-03,14.730000,14.820000,14.400000,14.630000,14.630000,19576600


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

InvalidConfigError: ignored

# Copilando as informações


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
from google.colab import drive
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive_google = GoogleDrive(gauth)
file_list = drive_google.ListFile({'q': "'1w7dABMRj_mwxWVYZyIjkAMoGycKl-kkE' in parents and trashed=false"}).GetList()
drive.mount('/drive')

pasta = '/drive/MyDrive/BASE_DADOS_FORMULA_MAGICA/'


Mounted at /drive


In [ ]:
def copilaArquvios(pasta, arquivoExemplo, arquivosCopilacao):
  dftotal = pd.read_excel(pasta + arquivoExemplo)
  if 'Unnamed: 0' in dftotal.columns:
    dftotal =dftotal.drop('Unnamed: 0', axis='columns', inplace=False)
  dftotal = pd.DataFrame(columns = dftotal.columns) 
  for file1 in file_list:
    if (arquivosCopilacao in file1['title'] and '.xlsx' in file1['title']):
      df = pd.read_excel(pasta + file1['title'])
      if 'Unnamed: 0' in df.columns:
        df =df.drop('Unnamed: 0', axis='columns', inplace=False)
      dftotal = dftotal.append(df)
  return dftotal

In [ ]:
df = copilaArquvios(pasta, 'dataset_11_11_2021.xlsx', 'dataset_')
df

,papel,cotacao,p_l,p_vp,psr,divyield,p_ativo,p_capgiro,p_ebit,p_ativ_circliq,...,ev_ebitda,mrg_ebit,mrg_liq,liq_corr,roic,roe,liq2meses,patrim_liq,divbrut__patrim,cresc_rec5a
0,CFLU4,1000.00,0.00,0.00,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,8.88,10.72,1.10,17.68,32.15,0.0,60351000,0.06,8.14
1,POPR4,10.17,0.00,0.00,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,8.66,5.65,1.08,15.25,19.93,0.0,545803000,0.82,30.93
2,MNSA3,0.42,0.00,0.00,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,-208.15,-362.66,3.63,-13.50,145.70,0.0,-9105000,-6.52,-41.11
3,CLAN3,0.00,0.00,0.00,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,-1.05,0.0,1012240000,0.00,-63.96
4,IVTT3,0.00,0.00,0.00,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,-0.40,0.0,1083050000,0.00,20.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,UBBR11,14.75,1201.81,3.91,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.0,10317200000,0.00,10.58
978,UBBR3,18.00,1466.61,4.77,0.000,0.00,0.000,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.0,10317200000,0.00,10.58
979,GLOB4,30.49,4873.56,8.59,1.512,0.00,1.389,-63.21,46.91,-4.59,...,28.77,3.22,0.03,0.96,3.90,0.18,0.0,5675000000,1.76,5.35
980,LWSA3,21.00,29339.40,4.20,17.828,0.13,2.946,7.34,392.97,11.98,...,98.08,4.54,0.06,3.82,1.33,0.01,138860000.0,2950560000,0.04,53.23


In [ ]:
df = copilaArquvios(pasta, 'filtro_final_11_11_2021.xlsx', 'filtro_final')
df

,papel,cotacao,p_l,p_vp,psr,divyield,p_ativo,p_capgiro,p_ebit,p_ativ_circliq,...,liq_corr,roic,roe,liq2meses,patrim_liq,divbrut__patrim,cresc_rec5a,rank_pl,rank_roe,total_rank
0,PSSA3,21.45,15.42,1.47,0.576,5.64,0.302,5.10,0.70,-1.94,...,1.10,54.25,9.56,48546200.0,9409950000,0.00,7.69,325,17,342
1,SYNE3,5.45,0.68,0.54,0.357,159.87,0.187,1.13,0.54,-1.03,...,4.43,40.55,79.85,2120670.0,1543340000,0.81,36.81,340,28,368
2,GOAU3,10.17,2.01,0.68,0.128,22.62,0.141,0.53,0.54,1.52,...,2.26,33.86,33.85,1138860.0,16259200000,0.77,19.54,336,47,383
3,GOAU4,11.03,2.18,0.74,0.139,20.85,0.153,0.58,0.58,1.65,...,2.26,33.86,33.85,85723300.0,16259200000,0.77,19.54,338,48,386
4,TASA4,16.20,2.94,2.12,0.692,9.82,0.911,4.09,2.14,9.11,...,1.50,53.73,72.20,14729400.0,926833000,0.64,40.47,392,18,410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,B3SA3,12.67,16.57,3.42,7.515,6.48,1.567,10.54,11.76,-11.16,...,1.59,21.30,20.64,664039000.0,22692700000,0.44,37.30,81,94,175
92,TFCO4,12.44,41.06,8.94,5.743,1.86,5.224,9.85,28.96,15.85,...,3.47,24.16,21.77,1995840.0,222248000,0.34,0.00,94,85,179
93,PRIO3,25.71,19.19,3.97,6.456,0.00,1.973,5.33,15.17,-76.66,...,4.46,22.17,20.69,661472000.0,5686920000,0.71,61.56,86,93,179
94,RAIZ4,6.50,31.59,6.63,0.624,0.00,0.894,40.42,19.69,-2.13,...,1.05,5.91,20.98,71015900.0,10152300000,2.96,0.00,93,92,185


In [ ]:
missing = df.isna().sum()
print(missing)

papel              0
cotacao            0
p_l                0
p_vp               0
psr                0
divyield           0
p_ativo            0
p_capgiro          0
p_ebit             0
p_ativ_circliq     0
ev_ebit            0
ev_ebitda          0
mrg_ebit           0
mrg_liq            0
liq_corr           0
roic               0
roe                0
liq2meses          0
patrim_liq         0
divbrut__patrim    0
cresc_rec5a        0
rank_pl            0
rank_roe           0
total_rank         0
dtype: int64


In [ ]:
df.drop('papel', axis='columns', inplace=True)
from sklearn.preprocessing import Normalizer
values = Normalizer().fit_transform(df.values) 
print(values)

[[2.27947168e-09 1.63866915e-09 1.56215542e-10 ... 3.45374497e-08
  1.80657429e-09 3.63440240e-08]
 [3.53129891e-09 4.40602432e-10 3.49890167e-10 ... 2.20301216e-07
  1.81424531e-08 2.38443669e-07]
 [6.25492028e-10 1.23622318e-10 4.18224758e-11 ... 2.06652233e-08
  2.89067112e-09 2.35558945e-08]
 ...
 [4.49062560e-09 3.35181273e-09 6.93418267e-10 ... 1.50211514e-08
  1.62438032e-08 3.12649546e-08]
 [6.40233344e-10 3.11153405e-09 6.53038011e-10 ... 9.16026169e-09
  9.06176426e-09 1.82220260e-08]
 [5.02234549e-08 1.57679040e-08 3.23626378e-09 ... 5.94283968e-08
  6.13662793e-08 1.20794676e-07]]


#Clusterização

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, n_init=10, max_iter=300)
y_pred = kmeans.fit_predict(values)

In [ ]:
from sklearn import metrics
labels = kmeans.labels_
silhouette = metrics.silhouette_score(values,labels,metric='euclidean')
print(silhouette) 

0.7926819047103522


In [ ]:
dbs = metrics.davies_bouldin_score(values, labels)
print(dbs)

In [ ]:
calinski = metrics.calinski_harabasz_score(values, labels) 
print(calinski)

In [ ]:
def clustering_algorithm(n_clusters, dataset):
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, max_iter=300)
    labels = kmeans.fit_predict(dataset)
    s = metrics.silhouette_score(dataset, labels, metric='euclidean')
    dbs = metrics.davies_bouldin_score(dataset, labels)
    calinski = metrics.calinski_harabasz_score(dataset, labels)
    return s, dbs, calinski

In [ ]:
s2, dbs2, calinski2 = clustering_algorithm(10, values)
print(s2, dbs2, calinski2)

0.6895889229644778 0.41529195509242867 31081.107069615577


In [ ]:
s2, dbs2, calinski2 = clustering_algorithm(5, values)
print(s2, dbs2, calinski2)

0.7926819047103522 0.39066534768162336 18420.612105458968


In [ ]:
df.count()

cotacao            4971
p_l                4971
p_vp               4971
psr                4971
divyield           4971
p_ativo            4971
p_capgiro          4971
p_ebit             4971
p_ativ_circliq     4971
ev_ebit            4971
ev_ebitda          4971
mrg_ebit           4971
mrg_liq            4971
liq_corr           4971
roic               4971
roe                4971
liq2meses          4971
patrim_liq         4971
divbrut__patrim    4971
cresc_rec5a        4971
rank_pl            4971
rank_roe           4971
total_rank         4971
dtype: int64

In [ ]:
import numpy as np
random_data = np.random.rand(4971,23)
s, dbs, calinski = clustering_algorithm(5, random_data)
print(s, dbs, calinski)
print(s2,dbs2, calinski2)

0.027745367562844257 4.194261497828589 117.25983011517023
0.7926819047103522 0.39066534768162336 18420.612105458968
